In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import numpy as np
import pandas as pd

from datetime import date, timedelta
import luigi
import os
import logging

from bt_ai.stable.data_input.user_trait_store import UserTraitsUnloadTask
from bt_ai.stable.data_input.dataframe import DataFrameTarget

from notebook_utils.logging import setup_logging  #pip3 install bt-notebook-utils
from notebook_utils.luigi import run_luigi_tasks

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

pd.options.display.max_columns = 999

In [4]:
luigi_planner_uri = 'http://localhost:8082'

logging_overrides = {
    'luigi-interface': logging.INFO,
}

LOG = logging.getLogger('jupyter')

In [5]:
class TrainingUsers(luigi.ExternalTask):
    path = luigi.Parameter()

    def output(self):
        root_path = os.path.dirname(self.path)
        filename = os.path.basename(self.path)

        return DataFrameTarget(
            root_path=root_path,
            column_names=[],
            filenames=[filename],
            field_sep=',',
            compute=True,
            compression=None
        )

In [15]:
users = TrainingUsers(
    path='s3://kalyan-sandbox/ml-dumps/zcode_project/large-scale/email_md5_df_sample200.csv'
    #path="/tmp/email_md5_sample.csv"
)

In [16]:
end_date = date(2021, 6, 25)
start_date = end_date - timedelta(days=3)
s3_bucket = 'drose-sandbox' # change this to point to your sandbox.
spark_job_name = 'drose_Sprint_PlaceIQ_Pyarrow'#change this to your name, so that you identify your launched cluster.

- 'm5.4xlarge' is not allowed in 1e aws_az

In [17]:
partition_date = date(2021, 6, 25)

user_traits = UserTraitsUnloadTask(
    site_id='sprint',
    partition_date=partition_date,
    spark_job_name=spark_job_name,
    instance_type='r5.4xlarge',
    core_count=48,
    use_spots=True,
    aws_az = 'us-east-1a',
    keep_alive=True,
    write_partitions=200,
    
    staging_bucket=s3_bucket,
    unload_bucket=s3_bucket,
    dependencies=[users]
)

In [18]:
# do not forget to initiat luigi from terminal by typing 'luigid'
run_luigi_tasks([user_traits], scheduler_uri=luigi_planner_uri)

Data for TrainingUsers(path=s3://kalyan-sandbox/ml-dumps/zcode_project/large-scale/email_md5_df_sample200.csv) does not exist (yet?). The task is an external data dependency, so it cannot be run from this luigi process.
